# Regression model: Predict Ames house prices

Estimate the value of houses in Ames, Iowa. This notebook builds and compares a univariate and multivariate regression model. The first uses the feature that is most strongly correlated with sale price, the second uses the two most strongest correlated features.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

### Importing and splitting data

In [ ]:
# import the data
data = pd.read_csv('data/AmesHousingNumData.csv')
data.head()

In [ ]:
# split data into input and target
target = data['SalePrice']
final_data = data.drop('SalePrice', axis=1)
final_data.head()

In [ ]:
# convert the pandas dataframes to numpy ndarrays
X_np = final_data.to_numpy()
y_np = target.to_numpy()

# split the data into 70% training and 30% testing
X_train, X_test, y_train, y_test = train_test_split(X_np, y_np, train_size=0.7, random_state=1265599650)

### Building the model

#### Visualizing a single feature 

In [ ]:
# make column vector out of row vector
y_train = y_train[..., None]

# make column vector for one feature for testing
X_single_feature = X_train[:, 1, None]

# retrieve name of said feature
feature_name = final_data.columns.values[1]

# plot the datapoints
plt.figure(figsize=(12,9))
plt.plot(X_single_feature, y_train, 'r.')
plt.title(f'Relation of {feature_name} and Sale Price')
plt.xlabel(f'{feature_name}')
plt.ylabel('Sale Price normalized')
plt.show()

#### Adding  $x_0$to the data

First an extra feature $x_0$ has to be added to the data. This is to make the computations easier.

In [ ]:
def add_x0(x):
    '''
    This function takes a column vector x, adds a column vector of ones in front of it, and returns the 
    combined matrix.
    '''
    # create new feature vector of ones in shape of number of samples * 1 column vector 
    x_zero = np.ones((len(x), 1), dtype = int)

    # combine the new and old feature vectors into matrix
    return np.hstack((x_zero, x))

# create feature with x0
X = add_x0(X_single_feature)


#### Defining the linear model


First, we implement **univariate linear regression** with the feature selected above. The system of linear equations looks like this:

$$x^1_0 \theta_0 + x^1_1 \theta_1 = h_{\theta}(x^1)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 = h_{\theta}(x^2)$$
$$x^3_0 \theta_0 + x^3_1 \theta_1 = h_{\theta}(x^3)$$ 
$$\dots$$
$$x^m_0 \theta_0 + x^m_1 \theta_1 = h_{\theta}(x^m)$$


x_0 is the column vector of ones that were added in front of the column vector of the selected feature. 

In terms of matrix multiplication:

$$ \left[\begin{array}{cccc}
x^1_0 & x^1_1 \\ 
x^2_0 & x^2_1 \\
x^3_0 & x^3_1 \\ 
\vdots & \vdots \\
x^m_0 & x^m_1 \\ 
\end{array} \right]
\left[\begin{array}{c} \theta_0 \\ \theta_1 \end{array} \right]
= \left[\begin{array}{c} h_{\theta}(x^1) \\ h_{\theta}(x^2) \\
h_{\theta}(x^3) \\ \vdots \\ h_{\theta}(x^m) \end{array} \right]$$


In [ ]:
def define_theta(X):
    '''
    Returns starting vector for theta with amount of features.
    '''
    return np.ones((X.shape[1], 1))


In [ ]:
def linear_model(X, theta):
    '''
    This function takes a matrix X and a vector theta and returns the resulting column vector of hypothesis
    values.
    '''
    return np.matmul(X, theta)

theta = define_theta(X)

# compute the linear model
h = linear_model(X, theta)

# plot the model results for just one of the features
plt.figure(figsize=(12,9))
plt.plot(X_single_feature, y_train, 'r.')
plt.plot(X_single_feature, h)
plt.title(f'Relation of {feature_name} and Sale Price')
plt.xlabel(f'{feature_name}')
plt.ylabel('Sale Price normalized')
plt.show()

### Optimizing theta

#### Defining linear cost

In [ ]:
def linear_cost(theta, X, y):
    '''
    This function returns the cost using matrix multiplication. 
    '''
    
    # m is identical to the number of rows in X
    m = X.shape[0]

    # compute error vector 
    residual = linear_model(X, theta) - y
    
    # compute cost vector  
    cost = np.matmul(residual.T, residual) / (2 * m)
    
    # convert the resulting 1 x 1 numpy matrix to an actual scalar 
    return np.squeeze(cost)

#### Defining gradient vector

In [ ]:
def gradient_vector(theta, X, y):
    '''
    This functions returns a (n + 1) * 1 column vector of partial derivatives. 
    '''
    
    # m is identical to the number of rows in X
    m = X.shape[0]
    
    # compute error vector
    error = linear_model(X, theta) - y 
    
    # compute matrix product of the error and X
    gradient_vector = np.matmul(X.T, error) / m
  
    return gradient_vector

In [ ]:
def gradient_descent(X, y, theta, alpha, thres=10**-6):
    '''
    This function returns a vector for theta for which the cost is the minimum. 
    '''
    # calculate cost with current theta
    previous_cost = linear_cost(theta, X, y)
    
    # set cost difference to amount bigger than the threshold and bigger than 0
    cost_difference = thres + 1
    
    # loop indefinitely until algorithm converges
    while cost_difference > thres: 
        
        # compute and update theta parameters in a single operation
        theta = theta - alpha * gradient_vector(theta, X, y)

        # calculate cost vector with new theta
        current_cost = linear_cost(theta, X, y)

        cost_difference = abs(current_cost - previous_cost)

        if current_cost > previous_cost:
            print('The algorithm is diverging.')
            return theta, previous_cost

        previous_cost = current_cost
    
    return theta, previous_cost

# find the theta vector that minimizes the cost function
theta_hat, cost = gradient_descent(X, y_train, theta, 0.9)
print("\nFound the following values for theta:")
print(theta_hat)
print(f'\nThe cost is {cost}.')

# plot the model results
print("\nResulting in the following model:")
h = linear_model(X, theta_hat)
plt.figure(figsize=(12,9))
plt.plot(X_single_feature, y_train, 'r.')
plt.plot(X_single_feature, h)
plt.title(f'Relation of {feature_name} and Sale Price')
plt.xlabel(f'{feature_name}')
plt.ylabel('Sale Price in logarithmic scale')
plt.show()

#### Visualizing multiple features

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

# make matrix for all features (two for now)
X_all_train = X_train[:, 0:]

#retrieve name of second feature
second_feature_name = final_data.columns.values[0]

#create 3d plot
ax = Axes3D(plt.figure(figsize=(12,9)))

# retrieve datapoints and plot
xs = X_all_train[:, 0]
ys = X_all_train[:, 1]
zs = y_train
ax.scatter(xs, ys, zs)

# set layout
ax.set_xlabel(f'{second_feature_name}', fontsize=14)
ax.set_ylabel(f'{feature_name}', fontsize=14)
ax.set_zlabel('Sale Price in logarithmic scale', fontsize=14)

plt.show()

#### Defining multivariate linear regression

Our model is a **multivariate linear** regression model which will look like the following:

$$x^1_0 \theta_0 + x^1_1 \theta_1 + \dots + x^1_n \theta_n = h_{\theta}(x^1)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 + \dots + x^2_n \theta_n = h_{\theta}(x^2)$$
$$x^2_0 \theta_0 + x^2_1 \theta_1 + \dots + x^2_n \theta_n = h_{\theta}(x^2)$$
$$\dots$$
$$x^m_0 \theta_0 + x^m_1 \theta_1 + \dots + x^m_n \theta_n = h_{\theta}(x^m)$$


In terms of matrix multiplication:

$$ \left[\begin{array}{cccc}
x_0^1 & x_1^1 & x_2^1 & \cdots & x_n^1 \\ 
x_0^2 & x_1^2 & x_2^2 & \cdots & x_n^2 \\ 
x_0^3 & x_1^3 & x_2^3 & \cdots & x_n^3 \\ 
\vdots & \vdots & \vdots & \ddots & \vdots \\
x_0^m & x_1^m & x_2^m & \cdots & x_n^m \\ 
\end{array} \right]
\left[\begin{array}{c} \theta_0 \\ \theta_1 \\
\theta_2 \\ \vdots \\ \theta_n \end{array} \right]
= \left[\begin{array}{c} h_{\theta}(x^1) \\ h_{\theta}(x^2) \\
h_{\theta}(x^3) \\ \vdots \\ h_{\theta}(x^m) \end{array} \right]$$

With *n* being the number of features and *m* the number of samples.

In [ ]:
# add x zero
X_train_x0 = add_x0(X_all_train)

theta = define_theta(X_train_x0)

# compute the linear model
h = linear_model(X_train_x0, theta)

theta_hat, cost = gradient_descent(X_train_x0, y_train, theta, 0.05)
print("\nFound the following values for theta:")
print(theta_hat)
print(f'\nThe cost is {cost}.')

In [ ]:
def gradient_descent_learning_curves(X_train, y_train, X_test, y_test, theta, alpha, thres):
    '''
    Implements gradient descent, stores training and testing cost at every step, and plots learning curves.
    '''
    # initialize number of iterations
    iteration = 0
    iterations = []
    
    # create lists for training and testing cost to keep track at each step
    current_costs_train = []
    current_costs_test = []

    # compute training cost with start theta
    previous_cost = linear_cost(theta, X_train, y_train)

    cost_diff = thres + 1

    while cost_diff > thres:

        iteration += 1
        iterations.append(iteration)
        
        # compute new theta
        theta = theta - alpha * gradient_vector(theta, X_train, y_train)
        
        # compute new training cost and append it to a list
        current_cost_train = linear_cost(theta, X_train, y_train)
        current_costs_train.append(current_cost_train)
        
        # compute new testing cost and append it to a list
        current_cost_test = linear_cost(theta, X_test, y_test)
        current_costs_test.append(current_cost_test)

        # compare the old cost vs the new cost
        cost_diff = previous_cost - current_cost_train

        # check if the algorithm is diverging
        if cost_diff < 0:
            print('The algorithm is diverging.')

        previous_cost = current_cost_train
        
    # plot learning curves
    plt.plot(iterations, current_costs_train, 'b-', label='Training')
    plt.plot(iterations, current_costs_test, 'r-', label='Testing')    
    plt.xlabel('Number of iterations')
    plt.ylabel('Cost')
    plt.legend()
    plt.show()
    
    return theta, current_cost_train, current_cost_test

# make matrix for all features 
X_all_test = X_test[:, 0:]

# add x zero
X_test_x0 = add_x0(X_all_test)

# make column vector out of row vector
y_test = y_test[..., None]

# initialize theta to a zero vector of the correct shape
theta = define_theta(X_train_x0)

# find the theta vector that minimizes the cost functions
plt.title('Learning curves of gradient descent with learning rate = 0.8, threshold = 10**-4\n')
theta_hat, cost_train, cost_test = gradient_descent_learning_curves(X_train_x0, y_train, X_test_x0, y_test, theta, 0.8, 10**-4)
print(f'\nThe training cost is {cost_train}. The testing cost is {cost_test}.')


#### Prediction accuracy

Taken from https://www.kaggle.com/kamalp0/ames-housing-regression-model

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
predictions = linear_model(X_test_x0, theta_hat)

# calculate average error
MSE = mean_squared_error(y_test, predictions)

# calculate standard deviation
RMSE = np.sqrt(MSE)

# calculate average price
avg_price = np.mean(target)
off_by = (RMSE)/ avg_price* 100

print(f'The test voor MSE for our linear model was {MSE}')
print(f'The test RMSE for our linear model {RMSE}')
print(f'The percentage our model was off compared to the real house price was:{off_by}')

### Sklearn regression model

In [ ]:
# Build a second linear regression model with the sklearn function
linear_model2 = LinearRegression()

linear_model2.fit(X_train, y_train)

predict = linear_model2.predict(X_test)

linear_test_mae = mean_absolute_error(y_test, predict)
linear_test_rmse = np.sqrt(mean_squared_error(y_test, predict))

print(f'The test MAE for our linear model {linear_test_mae}')
print(f'The test RMSE for our linear model {linear_test_rmse}')